In [165]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [166]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
from proyeccion_rdr.produccion.a04_ambulatorio import (
    obtener_distribucion_consultas,
    procesar_incidencias,
    leer_consultas_medicas,
    expandir_serie_rendimientos,
)

pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [167]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
casos_macroproceso_por_region, casos_macroproceso = features.leer_casos_macroprocesos(
    RUTA_ARCHIVOS
)

# Pone el indice en el DataFrame de los casos
casos_macroproceso = casos_macroproceso.reset_index().set_index(["Diagnostico", "tipo_paciente"])
casos_macroproceso_por_region = casos_macroproceso_por_region.reset_index().set_index(
    ["Diagnostico", "tipo_paciente"]
)

# Define las areas de influencia presenciales
AREAS_PRESENCIALES = [
    "Metropolitana de Santiago",
    "COMUNAS_SIN_SS_EN_RM",
    "SSMC",
    "SSMN",
    "SSMO",
    "SSMOC",
    "SSMS",
    "SSMSO",
    "Acotado por oferta",
]
casos_macroproceso_por_region["es_presencial"] = np.where(
    casos_macroproceso_por_region["Estrato"].isin(AREAS_PRESENCIALES),
    "Box Presencial",
    "Box de Telemedicina",
)

## Objetivo de este cuadernillo

En este cuadernillo se quieren estimar la cantidad de pacientes ambulatorios que asisitrán al INT para el 2035. Este insumo se utilizará para estimar la cantidad de Box Ambulatorios para cada especialidad. Por lo tanto, es necesario hacer los siguientes calculos:

1. Pacientes Ambulatorios
2. Boxes Ambulatorios

In [168]:
RUTA_INCIDENCIAS = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

(
    df_incidencias,
    diagnosticos_ingresados,
    casos_por_especialidad_long,
    casos_totales_por_especialidad_y_grupo,
    casos_a_hacerse_cargo_long,
    casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial,
    casos_a_hacerse_cargo_consolidados,
) = procesar_incidencias(
    RUTA_INCIDENCIAS, "consultas_medicas", casos_macroproceso_por_region, ANIOS_POBLACION
)

Iniciando proceso...
Cargando datos desde ../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx, hoja: consultas_medicas
Separando diagnósticos por especialidad...
Diagnósticos únicos ingresados: 49
Indexando por tipo de paciente y diagnóstico...
Uniendo casos por diagnóstico con macroprocesos...
Identificando casos duplicados...
Especialidades con diagnósticos duplicados: 

ESTAMENTO/ESPECIALIDAD
ANESTESIOLOGÍA                                                 [HOTI]
CIRUGÍA CARDIOVASCULAR                       [Q211, Q234, Q250, Q251]
CIRUGÍA PEDIÁTRICA                                       [K409, QLAP]
CIRUGÍA PLÁSTICA Y REPARADORA                                  [QLAP]
CIRUGÍA Y TRAUMATOLOGÍA BUCO MAXILOFACIAL                      [HEST]
DERMATOLOGÍA                                                   [DQUI]
OFTALMOLOGÍA                                                   [HEST]
OTORRINOLARINGOLOGÍA                                     [HOTI, J351]
TRA

## Test de Calidad

En este test se quiere saber si todas las trazadoras estan ingresadas en el macroproceso ambulatorio

In [169]:
# Lee la planilla de trazadoras y aisla
trazadoras_totales = pd.read_excel(RUTA_INCIDENCIAS, sheet_name="trazadoras")
trazadoras_totales = set(trazadoras_totales["Diagnostico"].str.split(" - ").str[0].unique())

In [170]:
# diagnosticos_sin_ingresar = trazadoras_totales - diagnosticos_ingresados
# if len(diagnosticos_sin_ingresar) > 0:
#     raise ValueError(f"Falta ingresar las trazadoras: {diagnosticos_sin_ingresar}")

## Lectura de performance historico de ambulatorio

In [171]:
# Lee las consultas medicas
RUTA_AMBULATORIO = "../data/raw/6_ambulatorio/df_procesada_consultas.csv"
consultas_medicas = leer_consultas_medicas(RUTA_AMBULATORIO)

C:\Users\jarojas\Desktop\Javier\SSMN\EPH - RDR\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a04_ambulatorio.py:90: DtypeWarning: Columns (19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})


In [172]:
# Obtiene resumen de consultas acumuladas en el periodo
agrupacion_acumulada = ["especialidad_agrupada"]
distribucion_consultas_medicas_acumuladas, consultas_medicas_por_paciente_acumuladas = (
    obtener_distribucion_consultas(consultas_medicas, agrupacion_acumulada)
)

# Aisla el nombre de la columna que tenga el 75% de las consultas
columna_estadistica_cantidad_consultas = distribucion_consultas_medicas_acumuladas.columns[
    distribucion_consultas_medicas_acumuladas.columns.str.contains("75%")
][0]

# Indica cuantas consultas por cada una de las especialidades
cantidad_consultas_medicas_a_ocupar = distribucion_consultas_medicas_acumuladas[
    columna_estadistica_cantidad_consultas
]

# Filtra las especialidades para solo dejar las que estan en la cartera de servicios
cantidad_consultas_medicas_a_ocupar = cantidad_consultas_medicas_a_ocupar[
    cantidad_consultas_medicas_a_ocupar.index.isin(
        casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial.index.get_level_values(0)
    )
]

## Definicion de desempeno para consultas sin produccion

In [173]:
# Define la cantidad de consultas que tendran las especialidades sin un rendimiento historico
consultas_sin_desempeno = pd.Series(
    {
        "CIRUGÍA CARDIOVASCULAR": None,
        "CIRUGÍA VASCULAR PERIFÉRICA": None,
    }
)

# Concatena las especilidades que si tienen desempeno con las que no las tienen
cantidad_consultas_medicas_a_ocupar = pd.concat(
    [cantidad_consultas_medicas_a_ocupar, consultas_sin_desempeno]
)

In [174]:
# Indica la cantidad de consultas que tendran la telemedicina
grupos_de_pacientes = [1, 2]
valores_es_presencial = ["Box Presencial", "Box de Telemedicina"]

# Expande la cantidad de consultas segun grupo de pacientes y valores presenciales
cantidad_consultas_medicas_a_ocupar = expandir_serie_rendimientos(
    cantidad_consultas_medicas_a_ocupar, grupos_de_pacientes, valores_es_presencial
)

In [175]:
TIEMPO_CONSULTA = {
    "15 minutos": 0.25,
    "30 minutos": 0.5,
    "45 minutos": 0.75,
    "60 minutos": 1,
    "20 minutos": 0.333,
    "40 minutos": 0.666,
}

rendimientos_reales = pd.Series(
    {
        "ANESTESIOLOGÍA": TIEMPO_CONSULTA["20 minutos"],
        "CARDIOLOGÍA": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA PEDIÁTRICA": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA PLÁSTICA Y REPARADORA": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA Y TRAUMATOLOGÍA BUCO MAXILOFACIAL": TIEMPO_CONSULTA["30 minutos"],
        "DERMATOLOGÍA": TIEMPO_CONSULTA["20 minutos"],
        "ENDOCRINOLOGÍA PEDIÁTRICA": TIEMPO_CONSULTA["40 minutos"],
        "ENFERMEDADES RESPIRATORIA PEDIÁTRICAS": TIEMPO_CONSULTA["30 minutos"],
        "GASTROENTEROLOGÍA PEDIÁTRICA": TIEMPO_CONSULTA["40 minutos"],
        "GENÉTICA CLÍNICA": TIEMPO_CONSULTA["60 minutos"],
        "GINECOLOGÍA PEDIÁTRICA Y DE LA ADOLESCENCIA": TIEMPO_CONSULTA["30 minutos"],
        "HEMATOLOGÍA": TIEMPO_CONSULTA["40 minutos"],
        "INFECTOLOGÍA": TIEMPO_CONSULTA["40 minutos"],
        "INMUNOLOGÍA": TIEMPO_CONSULTA["40 minutos"],
        "NEFROLOGÍA PEDIÁTRICA": TIEMPO_CONSULTA["40 minutos"],
        "NEUROCIRUGÍA": TIEMPO_CONSULTA["30 minutos"],
        "NEUROLOGÍA PEDIATRICA": TIEMPO_CONSULTA["45 minutos"],
        "OFTALMOLOGÍA": TIEMPO_CONSULTA["20 minutos"],
        "OTORRINOLARINGOLOGÍA": TIEMPO_CONSULTA["20 minutos"],
        "PEDIATRÍA": TIEMPO_CONSULTA["30 minutos"],
        "REUMATOLOGÍA": TIEMPO_CONSULTA["40 minutos"],
        "TRAUMATOLOGÍA Y ORTOPEDIA": TIEMPO_CONSULTA["30 minutos"],
        "UROLOGÍA": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA CARDIOVASCULAR": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA VASCULAR PERIFÉRICA": TIEMPO_CONSULTA["30 minutos"],
    }
)

# Expande los rendimientos
rendimientos_reales = expandir_serie_rendimientos(
    rendimientos_reales, grupos_de_pacientes, valores_es_presencial
)

## Estimacion de Consultas al 2035

In [176]:
# Obtiene la cantidad de horas laborales
horas_laborales = features.calcular_horas_laborales(2017, 2035, 12)

Horas laborales por año calculadas:
+---+------+---------------------------------+
|   | anio | horas_laborales_solo_semana_12h |
+---+------+---------------------------------+
| 0 | 2017 |              2976               |
| 1 | 2018 |              2964               |
| 2 | 2019 |              2988               |
| 3 | 2020 |              3024               |
| 4 | 2021 |              3012               |
+---+------+---------------------------------+



In [177]:
# Multiplica los casos de area de influencia
consultas_proyectadas = casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial.mul(
    cantidad_consultas_medicas_a_ocupar, axis=0
).dropna(axis=0, how="all")

# Multiplica la cantidad de consultas por el tiempo que requerira cada consulta
tiempo_consultas = consultas_proyectadas.mul(rendimientos_reales, axis=0).dropna(axis=0, how="all")

# Divide la cantidad de consultas por especialidad, por el rendimiento de cada box
boxes_proyectados = tiempo_consultas.div(horas_laborales, axis=1)

In [178]:
# Obtiene el resumen de Boxes por estamnento
boxes_consolidados = (
    boxes_proyectados.reset_index()
    .groupby("ESTAMENTO/ESPECIALIDAD")
    .sum()["2035"]
    .sort_values(ascending=False)
)

# Agrega el total
boxes_consolidados["Total"] = boxes_consolidados.sum()

In [179]:
display(boxes_consolidados)

ESTAMENTO/ESPECIALIDAD
ENFERMEDADES RESPIRATORIA PEDIÁTRICAS            9.02145
OTORRINOLARINGOLOGÍA                             7.24285
ANESTESIOLOGÍA                                   4.93394
ENDOCRINOLOGÍA PEDIÁTRICA                       4.767862
TRAUMATOLOGÍA Y ORTOPEDIA                       4.182995
NEFROLOGÍA PEDIÁTRICA                           3.509824
PEDIATRÍA                                       1.816831
INFECTOLOGÍA                                    1.691249
CIRUGÍA PEDIÁTRICA                              1.389873
HEMATOLOGÍA                                     1.161092
CIRUGÍA PLÁSTICA Y REPARADORA                    1.11173
CIRUGÍA Y TRAUMATOLOGÍA BUCO MAXILOFACIAL       0.684993
REUMATOLOGÍA                                    0.635216
CARDIOLOGÍA                                     0.603522
GASTROENTEROLOGÍA PEDIÁTRICA                    0.558359
OFTALMOLOGÍA                                    0.465548
INMUNOLOGÍA                                      0.39701
NEUROLOG

In [180]:
# Obtiene la cantidad de boxes medicos y de telemedicina
boxes_presencial_y_tele = (
    boxes_proyectados.reset_index().groupby("es_presencial")[ANIOS_POBLACION].sum()
)

In [181]:
boxes_presencial_y_tele

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
es_presencial,,,,,,,,,,,,,,,,,,,
Box Presencial,32.158787,33.320141,34.046069,34.583839,35.268564,35.755564,36.506713,36.534795,36.820144,36.725818,36.84078,37.357442,37.694398,37.414575,37.135772,36.698803,36.954659,37.459888,37.524953
Box de Telemedicina,8.028012,8.014455,7.950391,7.876684,7.933707,7.999547,8.129967,8.105529,8.142134,8.082255,8.034128,8.035412,7.984284,7.794028,7.623856,7.437359,7.391869,7.399375,7.338524


In [182]:
# Define columnas que se utilizar para generar resumen MINSAL
cols_indice = ["ESTAMENTO/ESPECIALIDAD", "Grupos de Pacientes", "es_presencial"]

# Genera un DataFrame de las trazadoras con presencialidad
df_incidencias_resumen = df_incidencias.copy()
df_incidencias_resumen["es_presencial"] = "Box Presencial, Box de Telemedicina"
df_incidencias_resumen["es_presencial"] = df_incidencias_resumen["es_presencial"].str.split(", ")
df_incidencias_resumen = df_incidencias_resumen.explode("es_presencial")
df_incidencias_resumen = df_incidencias_resumen.set_index(cols_indice)

## Resumen MINSAL

In [183]:
# Obtiene el resumen MINSAL
resumen_MINSAL = pd.DataFrame(
    {
        "tipo_paciente": df_incidencias_resumen["tipo_paciente"],
        "Diagnostico": df_incidencias_resumen["Diagnostico"],
        "Areas de Influencia a atender presencial": str(AREAS_PRESENCIALES),
        "numero_de_pacientes_totales_2035": casos_totales_por_especialidad_y_grupo["2035"],
        "% de los pacientes a atender": df_incidencias_resumen["% de los pacientes a atender"],
        "explicacion_pacientes": df_incidencias_resumen["Explicación Pacientes"],
        "casos_a_hacerse_cargo_2035": casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial[
            "2035"
        ],
        "consultas_por_paciente_75%": cantidad_consultas_medicas_a_ocupar,
        "consultas_proyectadas_2035": consultas_proyectadas["2035"],
        "horas_por_consultas": rendimientos_reales,
        "horas_consultas_2035": tiempo_consultas["2035"],
        "horas_laborales_2035": horas_laborales["2035"],
        "boxes_proyectados_2035": boxes_proyectados["2035"],
    }
)

resumen_MINSAL = resumen_MINSAL.dropna().reset_index(level=[1, 2])

In [184]:
a_guardar = {
    "resumen_MINSAL": resumen_MINSAL,
    "boxes_consolidados": boxes_consolidados,
    "trazadoras_ambulatorio": df_incidencias,
    "casos_macroprocesos_por_region": casos_macroproceso_por_region.reset_index(),
    "casos_por_esp_long": casos_por_especialidad_long.reset_index(),
    "casos_a_hacerse_cargo_long": casos_a_hacerse_cargo_long,
    "casos_por_esp_grupo_y_pres": casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial,
    "casos_consolidados": casos_a_hacerse_cargo_consolidados,
    "metricas_historicas_por_espec": distribucion_consultas_medicas_acumuladas,
    "rendimientos_utilizados": rendimientos_reales,
    "consultas_medicas_proyectadas": consultas_proyectadas,
    "tiempo_por_especialidad": tiempo_consultas,
    "horas_laborales": horas_laborales,
    "boxes_medicos_proyectados": boxes_proyectados,
    "boxes_presenciales_y_tele": boxes_presencial_y_tele,
}

with pd.ExcelWriter("../data/interim/3.0_estimacion_boxes_medicos_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)